In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")
import pandas as pd

In [ ]:
plt.rcParams['figure.figsize'] = (7,5)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc

In [ ]:
from scipy.stats import beta
from scipy.stats import f

In [ ]:
from itertools import cycle
from pca import pca

In [ ]:
# Machine Learning Modules
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
print("Pandas version: ", pd.__version__)
print("Seaborn version: ", sns.__version__)

Dataset
The "Admission Prediction" dataset is used for prediction of admission to postgraduate program using below mentioned data vairables.
The data consists of ~400 observations. Every observation is described by 7 feature columns which can be used for prediction.
GRE Score: Scores from the Graduate Record Examinations, an important factor in graduate admissions.
TOEFL Score: Test scores from the Test of English as a Foreign Language, indicating the applicant's proficiency in English.
University Rating: A rating of the university that might correlate with the competitiveness or prestige of the program.
SOP (Statement of Purpose): A qualitative measure of the applicant's statement of purpose, reflecting preparedness and intent.
LOR (Letter of Recommendation): Strength of the letters of recommendation.
CGPA: Undergraduate cumulative grade point average, a critical indicator of academic performance.
Research: A binary indicator of whether the applicant has research experience, which can be crucial for research-oriented programs.
Chance of Admit
The original dataset can be found at - https://www.kaggle.com/datasets/akshaydattatraykhare/data-for-admission-in-the-university

In [ ]:
# loads csv file into pandas dataframe
df = pd.read_csv('https://raw.githubusercontent.com/VishnuSharan-T/INSE6220/main/Admission-prediction.csv')
df = df.drop(columns=['Serial No.'])
# Renaming & converting the CGPA to percentage for convenience
df.rename(columns={'CGPA': 'Percentage'}, inplace=True)
df['Percentage'] = df['Percentage'].apply(lambda x: int(x * 10 * 9.5 / 10))
# Converting the Chance of Admit value to an integer value
df['Chance of Admit '] = df['Chance of Admit '].apply(lambda x: int(x * 100))
df['SOP'] = df['SOP'].apply(lambda x: int(x))
df['LOR '] = df['LOR '].apply(lambda x: int(x))

In [ ]:
print(df.head(n=30))

In [ ]:
df.info()

In [ ]:
print(len(df.index))
print("Number of duplicated rows is: ", df.duplicated().sum())
print("Number of rows with NaNs is: ", df.isna().any(axis=1).sum())

In [ ]:
sns.pairplot(df, hue='Chance of Admit ')  # Visualize data distribution using pair plot
plt.ylabel('')
plt.show()

In [ ]:
Y = df['Chance of Admit ']
target = df['Chance of Admit '].to_numpy()

In [ ]:
X = df.drop(columns=['Chance of Admit ']) #drop column class to standardize data
print(X.head(10))
print(X.describe().transpose)

In [ ]:
Xs = StandardScaler().fit_transform(X)                   #This is done to retain the column names after standardization
df = pd.DataFrame(Xs)
df.columns = X.columns
observations = list(df.index)
print(observations)
variables = list(df.columns)
print(variables)

In [ ]:
pca = PCA()
pca.fit(df)
Z = pca.fit_transform(df)

In [ ]:
# 1(Alpha)->0,  2(Beta)->1, 3(Gaama)->2
idx_Alpha = np.where(Y == 0)
idx_Beta = np.where(Y == 1)
idx_Gaama = np.where(Y == 2)

In [ ]:
plt. figure()
plt.scatter(Z[idx_Alpha,0], Z[idx_Alpha,1], c='r', label='Alpha (0)')
plt.scatter(Z[idx_Beta,0], Z[idx_Beta,1], c='g', label='Beta (1)')
plt.scatter(Z[idx_Gaama,0], Z[idx_Gaama,1], c='b', label='Gaama (2)')

In [ ]:
plt.legend()
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

In [ ]:
A = pca.components_.T
plt.figure(figsize=(8, 4))
plt.scatter(A[:,0],A[:,1],c='r')
plt.xlabel('Z1')
plt.ylabel('Z2')
for label, x, y in zip(variables, A[:, 0], A[:, 1]):
  plt.annotate(label, xy=(x, y), xytext=(-2, 2))
  textcoords='offset points',
  ha='right',
  va='bottom',
  bbox = dict(boxstyle='round', pad = 0.5, fc = 'yellow', alpha = 0.5)
  arrowprops = dict(arrowstyle='->', connectionstyle='arc3, rad=0')
plt.show()

In [ ]:
#Eigen Values
A = pca.components_.T
Lambda = pca.explained_variance_
x = np.arange(len(Lambda)) + 1
plt.figure(figsize=(15,10))
plt.plot(x, Lambda/sum(Lambda), 'ro-', lw=3)
plt.show()

In [ ]:
#Explained Variance
ell = pca.explained_variance_ratio_
ind = np.arange(len(ell))
plt.figure(figsize=(15,10))
plt.bar(ind, ell, align='center', alpha=0.5)
plt.plot(np.cumsum(ell))
plt.show()

In [ ]:
#Biplot
A1 = A[:,0]
A2 = A[:,1]
Z1 = Z[:,0]
Z2 = Z[:,1]

In [ ]:
plt.figure(figsize=(15,10))
plt.xlabel('Z1')
plt.ylabel('Z2')
for i in range(len(A1)):
    plt.arrow(0,0, A1[i]*max(Z1), A2[i]*max(Z2), color = 'k', width = 0.0005, head_width = 0.0025)
    plt.text(A1[i]*max(Z1)*1.2, A2[i]*max(Z2)*1.2, df.columns[i], color = 'k')
plt.scatter(Z[idx_Alpha,0], Z[idx_Alpha,1], c='r', label='Alpha (0)')
plt.scatter(Z[idx_Beta,0], Z[idx_Beta,1], c='g', label='Beta (1)')
plt.scatter(Z[idx_Gaama,0], Z[idx_Gaama,1], c='b', label='Gaama (2)')
plt.show()

In [ ]:
Y.value_counts().plot(kind='bar', rot=0)
plt.xlabel('Class')
plt.ylabel('Count')
plt.show()

In [ ]:
ax = plt.figure()
ax = sns.boxplot(data=df, orient="v", palette="Set2")
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
plt.show()

In [ ]:
ax = plt.figure()
ax = sns.boxplot(data=df, orient="v", palette="Set2")
ax = sns.stripplot(data=df, color=".25")
ax.set_xticklabels(ax.get_xticklabels(),rotation=45);
plt.show()

In [ ]:
print(df.describe())

In [ ]:
dfc = df - df.mean() #centered data
ax = sns.heatmap(dfc.cov(), cmap='RdYlGn_r', linewidths=0.5, annot=True, cbar=False, square=True)
plt.yticks(rotation=0)
ax.tick_params(labelbottom=False,labeltop=True)
ax.set_xticklabels(ax.get_xticklabels(),rotation=0);
plt.show()

In [ ]:
#Using PCA Library
from pca import pca
# Initialize and keep all PCs
model = pca()
# Fit transform
out = model.fit_transform(df)

In [ ]:
# Print the top features. The results show that f1 is best, followed by f2 etc
print(out['PC'])
model.scatter(label = True, legend = False)
plt.show()

In [ ]:
A = out['loadings'].T
print(A)
model.biplot3d(color_arrow='k')
plt.show()

In [ ]:
comps = pd.DataFrame(A, columns = variables)
ax = sns.heatmap(comps,cmap='RdYlGn_r', linewidths=0.5, annot=True, cbar=True, square=True)
ax.tick_params(labelbottom=False,labeltop=True)
plt.xticks(rotation=90)
plt.title('Principal components')
plt.show()

In [ ]:
print(f'PC1:{A1}')
print(f'PC2:{A2}')

Test-Train Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, Y, test_size=0.3, random_state=0)
print(f'Train Dataset Size: {X_train.shape[0]}')
print(f'Test Dataset Size: {X_test.shape[0]}')

In [ ]:
Z_train, Z_test, zy_train, zy_test = train_test_split(Z, Y, test_size=0.3, random_state=0)
Z12_train, Z12_test, z12y_train, z12y_test = train_test_split(Z[:,:2], Y, test_size=0.3, random_state=0)

In [ ]:
# Define the evaluation metric
scoring = ['f1_macro']

In [ ]:
# NV classifier
gnb = GaussianNB()

In [ ]:
datasets = [('FULL DATA', X_train, y_train, X_test, y_test), ('Z', Z_train, zy_train, Z_test, zy_test),
            ('Z12', Z12_train, z12y_train, Z12_test, z12y_test)]
for i, (name, Xtr, ytr, Xtst, ytst) in enumerate(datasets):
    gnb.fit(Xtr, ytr)
    y_pred = gnb.predict(Xtst)
    gnb_score = gnb.score(Xtst, ytst)

    # Classification Report
    print(f'DATASET: {name}')
    print('Classification Report:')
    print(classification_report(ytst, y_pred, digits=3))

    # Confusion Matrix
    cm_gnb = confusion_matrix(y_true=ytst, y_pred=y_pred)
    ax = sns.heatmap(cm_gnb, cmap='RdYlGn_r', linewidths=0.5, annot=True, square=True)
    plt.yticks(rotation=0)
    ax.tick_params(labelbottom=False, labeltop=True)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
    # plt.title('Naive Bayes Confusion Matrix')
    plt.show()

    # ADAPTED FROM: https://scikit-learn.org/stable/auto_examples/ensemble/plot_voting_decision_regions.html#sphx-glr-auto-examples-ensemble-plot-voting-decision-regions-py
    if name == 'Z12':
        # Plotting decision regions
        x_min, x_max = Xtr[:, 0].min() - 1, Xtr[:, 0].max() + 1
        y_min, y_max = Xtr[:, 1].min() - 1, Xtr[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01), np.arange(y_min, y_max, 0.01))

        Z = gnb.predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)

        plt.contourf(xx, yy, Z, alpha=0.4)
        plt.scatter(Xtr[:, 0], Xtr[:, 1], c=ytr, s=20, edgecolor="k", label='Train Set')
        plt.scatter(Xtst[:, 0], Xtst[:, 1], c=ytst, marker='^', s=20, edgecolor="k", label='Test Set')
        plt.xlabel('')
        plt.ylabel('')
        plt.legend()
        plt.show()

        print(np.where(ytst != y_pred))

In [ ]:
#KNN
# Hyperparameter grid search for k
param_grid = {'n_neighbors': [2, 4, 8, 16, 32]}
knn = KNeighborsClassifier()
grid_search = GridSearchCV(knn, param_grid, cv=5)

In [ ]:
# Find best k
knn_full_data = grid_search.fit(X_train, y_train)
knn_Z = grid_search.fit(Z_train, zy_train)
knn_Z12 = grid_search.fit(Z12_train, z12y_train)

In [ ]:
# Get best k
print('Grid Search Results:')
k_full_data = knn_full_data.best_params_
k_Z = knn_Z.best_params_
k_Z12 = knn_Z12.best_params_
print(f'k_full_data: {k_full_data}\nk_Z: {k_Z}\nk_Z12: {k_Z12}')

In [ ]:
# Apply best k
knn = KNeighborsClassifier(n_neighbors=k_full_data.get('n_neighbors'))
scores_knn_full_data = cross_validate(knn, X_train, y_train, cv=5, scoring=scoring)
scores_knn_Z = cross_validate(knn, Z_train, zy_train, cv=5, scoring=scoring)
scores_knn_Z12 = cross_validate(knn, Z12_train, z12y_train, cv=5, scoring=scoring)

In [ ]:
knn_scores_dict = {}
for i in ['fit_time', 'test_f1_macro']:
    knn_scores_dict["knn_full_data " + i] = scores_knn_full_data[i]
    knn_scores_dict["knn_Z  " + i] = scores_knn_Z[i]
    knn_scores_dict["knn_Z12 " + i] = scores_knn_Z12[i]

In [ ]:
knn_scores_data = pd.DataFrame(knn_scores_dict).T
# knn_scores_data['avgs'] = knn_scores_data.mean(axis=1)
print(f'{knn_scores_data}\n')

In [ ]:
datasets = [('FULL DATA', X_train, y_train, X_test, y_test), ('Z', Z_train, zy_train, Z_test, zy_test),
            ('Z12', Z12_train, z12y_train, Z12_test, z12y_test)]
for i, (name, Xtr, ytr, Xtst, ytst) in enumerate(datasets):
    # Apply on train-test split
    knn.fit(Xtr, ytr)
    y_pred = knn.predict(Xtst)
    knn_score = knn.score(Xtst, ytst)
    # print(f'\nTest Set Accuracy: {knn_score:.3f}')

    # Classification Report
    print(f'DATASET: {name}')
    print('Classification Report:')
    print(classification_report(ytst, y_pred, digits=3))

    # Confusion Matrix
    cm_knn = confusion_matrix(y_true=ytst, y_pred=y_pred)
    ax = sns.heatmap(cm_knn, cmap='RdYlGn_r', linewidths=0.5, annot=True, square=True)
    plt.yticks(rotation=0)
    ax.tick_params(labelbottom=False, labeltop=True)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
    # plt.title('KNN Confusion Matrix')
    plt.show()

    # ADAPTED FROM: https://scikit-learn.org/stable/auto_examples/ensemble/plot_voting_decision_regions.html#sphx-glr-auto-examples-ensemble-plot-voting-decision-regions-py
    if name == 'Z12':
        # Plotting decision regions
        x_min, x_max = Xtr[:, 0].min() - 1, Xtr[:, 0].max() + 1
        y_min, y_max = Xtr[:, 1].min() - 1, Xtr[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01), np.arange(y_min, y_max, 0.01))

        Z = knn.predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)

        plt.contourf(xx, yy, Z, alpha=0.4)
        plt.scatter(Xtr[:, 0], Xtr[:, 1], c=ytr, s=20, edgecolor="k", label='Train Set')
        plt.scatter(Xtst[:, 0], Xtst[:, 1], c=ytst, marker='^', s=20, edgecolor="k", label='Test Set')
        plt.xlabel('')

        plt.ylabel('')

        plt.legend()
        plt.show()

        # print(np.where(ytst != y_pred))

ADAPTED FROM: https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html#sphx-glr-auto-examples-model-selection-plot-roc-py

In [ ]:
datasets = [('FULL DATA', X_train, y_train, X_test, y_test), ('Z', Z_train, zy_train, Z_test, zy_test), ('Z12', Z12_train, z12y_train, Z12_test, z12y_test)]
for i, (name, X_tr, y_tr, X_tst, y_tst) in enumerate(datasets):
  # Binarize the labels
  y_train = label_binarize(y_tr, classes=[0, 1, 2])
  y_test = label_binarize(y_tst, classes=[0, 1, 2])
  n_classes = y_train.shape[1]
  print(f'DATASET: {name}')

  list_algos = [gnb, knn]
  algo_name = ['Naive Bayes', 'K-Nearest Neighbors']
  for i, (algo, algo_name) in enumerate(zip(list_algos, algo_name)):
    classifier = OneVsRestClassifier(algo)
    y_pred = classifier.fit(X_tr, y_train).predict(X_tst)
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:,i], y_pred[:,i], drop_intermediate=False)
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_pred.ravel(), drop_intermediate=False)
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    fig, ax = plt.subplots()

    plt.plot(fpr["micro"], tpr["micro"], label=f'micro-average ROC curve (area = {roc_auc["micro"]:0.2f})', color="deeppink", linestyle=':')
    plt.plot(fpr["macro"], tpr["macro"], label=f'macro-average ROC curve (area = {roc_auc["macro"]:0.2f})', color="navy", linestyle=':')

    colors = cycle(['c', 'm', 'r'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i],tpr[i], color=color,label=f'ROC curve of class {i} (area = {roc_auc[i]:0.2f})')

    plt.plot([0, 1], [0, 1], "k--")
    plt.xlim([-0.1, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f'{algo_name}')
    plt.legend()
    plt.show()

In [ ]:
# ADAPTED FROM: https://matplotlib.org/3.1.1/gallery/lines_bars_and_markers/barchart.html
def autolabel(rects):
    for r in rects:
        height = r.get_height()
        ax.annotate(f'{height}', xy=(r.get_x() + r.get_width() / 2, height), xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')

In [ ]:
n_groups = 3
ind = np.arange(n_groups)

In [ ]:
# F1 Scores from above
NB = (0.900, 0.060, 0.093)
KNN = (0.167, 0.073, 0.087)

In [ ]:
# create plot
fig, ax = plt.subplots(figsize=(10,7))
index = np.arange(n_groups)
bar_width = 0.20
opacity = 0.8

In [ ]:
rects1 = plt.bar(index, NB, bar_width, alpha=opacity, color='b', label='Naive Bayes')
rects2 = plt.bar(index + bar_width, KNN, bar_width, alpha=opacity, color='y', label='K-Nearest Neighbors')

In [ ]:
ax.set_xlabel('Data Set')
ax.set_ylabel('Macro-F1 Scores')
#plt.title(f'')
plt.xticks(index + bar_width, ('Original Data', 'All PCs', 'Two PCs'))
plt.legend(loc="upper right")

In [ ]:
autolabel(rects1)
autolabel(rects2)

In [ ]:
plt.tight_layout()
plt.show()